# Compare EC profit redistribution

This notebook aims at comparing the profit redistribution for an Energy Community
using the package EnergyCommunity.jl and TheoryOfGames.jl

## Configuration

### Paths

In [1]:
input_file = joinpath(@__DIR__, "../data/energy_community_model.yml")  # Input file

output_file_isolated = joinpath(@__DIR__, "../results/output_file_NC.xlsx")  # Output file - model users alone
output_plot_isolated = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_NC.png")  # Output png file of plot - model users alone

output_file_combined = joinpath(@__DIR__, "../results/output_file_EC.xlsx")  # Output file - model Energy community
output_plot_combined = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_EC.pdf")  # Output png file of plot - model energy community

output_plot_sankey_agg = joinpath(@__DIR__, "../results/Img/sankey_EC.png")  # Output plot of the sankey plot related to the aggregator case
output_plot_sankey_noagg = joinpath(@__DIR__, "../results/Img/sankey_NC.png")  # Output plot of the sankey plot related to the no aggregator case

enum_mode_file = "enum_mode_datasest.jld2"  # file used to store the enumerative results
total_results_file = "total_results_file_pmode0_psearch200_modifyobjstop_NC.jld2"  # file to store all major results
latex_output = "latex_output_pmode0_psearch200_modifyobjstop_NC.txt"

overwrite_files = true  # when true, output files are overwritten

true

### Import libraries

In [26]:
using Revise
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using TheoryOfGames
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings

### Solver configurations

In [3]:
# General optimizer
OPTIMIZER = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0, "Threads"=>10)

# Optimizer for row-generation techniques, used in the IterMode of TheoryOfGames.jl
OPTIMIZER_ROW_GENERATION = optimizer_with_attributes(Gurobi.Optimizer,
    "OutputFlag"=>1,
    "LogToConsole"=>0,
    "MIPGap"=>0.1,
    # "MIPFocus"=>1,
    "TimeLimit"=>1000,
    "LogFile"=>"gurobi_pmode0_psearch200_modifyobjstop_NC.log",
    "Threads"=>10,
    # "NoRelHeurTime"=>10,
    "PoolSolutions"=>200,
    "PoolSearchMode"=>0,
)

MathOptInterface.OptimizerWithAttributes(Gurobi.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("OutputFlag") => 1, MathOptInterface.RawOptimizerAttribute("LogToConsole") => 0, MathOptInterface.RawOptimizerAttribute("MIPGap") => 0.1, MathOptInterface.RawOptimizerAttribute("TimeLimit") => 1000, MathOptInterface.RawOptimizerAttribute("LogFile") => "gurobi_pmode0_psearch200_modifyobjstop_NC.log", MathOptInterface.RawOptimizerAttribute("Threads") => 10, MathOptInterface.RawOptimizerAttribute("PoolSolutions") => 200, MathOptInterface.RawOptimizerAttribute("PoolSearchMode") => 0])

### Energy Community options

In [4]:
NO_AGG_GROUP = GroupNC();  # type of aggregation when the Aggregator does not belong to the coalition.
                            # options: GroupANC() or GroupNC()
BASE_GROUP = GroupNC();     # base type of aggregation (it shall be GroupNC)

## General

In [5]:
ENV["COLUMNS"] = 1000  # to print more columns in the output

1000

## Optimize the Energy Community in the COperative configuration

In [6]:
# Read data from excel file
ECModel = ModelEC(input_file, EnergyCommunity.GroupCO(), OPTIMIZER)

# Reset the user set to use all stored users (10)
reset_user_set!(ECModel)
# set_user_set!(ECModel, ["user$id" for id=1:8])

# Build the model
build_model!(ECModel)

# Optimize the model
optimize!(ECModel)

Academic license - for non-commercial use only - expires 2023-02-04


An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"]


Solved model


## Calculate the reward distribution functions by enumerative techniques (EnumMode in TheoryOfGames.jl)

### Create enumerative mode

Calculation of the enumerative mode used to compute several reward distribution functions

In [ ]:
if !isfile(total_results_file) || overwrite_files
    tick()
    enum_mode = EnumMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP)
    time_elapsed_enum=tok()
    println("EnumMode calculated with elapsed time [min]: $(time_elapsed_enum/60)")
else
    time_elapsed_enum=0.0
end

Save enum mode

In [8]:
if !isfile(total_results_file) || overwrite_files
    save(enum_mode_file, enum_mode)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233


Load EnumMode (when needed)

In [9]:
enum_mode = load(enum_mode_file, EnumMode())

EnumMode(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"], Dict{Set{Any}, Float64}(Set(["user4", "user7", "user9", "user2", "user6"]) => 0.0, Set(["user4", "user7", "EC", "user5", "user1", "user10"]) => 22063.414377433248, Set(["user7", "user9", "user6", "user3", "user10"]) => 0.0, Set(["user4", "user3", "user10"]) => 0.0, Set(["user7", "user8", "user5", "user1", "user6", "user10"]) => 0.0, Set(["user8", "user9", "user2", "user6", "user10"]) => 0.0, Set(["user9", "user2", "user6", "user3"]) => 0.0, Set(["user4", "user7", "user6", "user3", "user10"]) => 0.0, Set(["EC", "user8", "user9", "user2", "user1", "user3", "user10"]) => 86831.98882880434, Set(["user7", "EC", "user5", "user6", "user3", "user10"]) => 173419.66703306255…))

### Calculate reward distribution using EnumMode

In [ ]:
tick()
shapley_dist_enum = shapley_value(enum_mode)  # shapley value
time_elapsed_shapley_enum=tok()

tick()
nucleolus_dist_enum, n_iterations_nucleolus_enum, model_nucleolus_enum = nucleolus(enum_mode, OPTIMIZER; raw_outputs=true)  # nucleolus
time_elapsed_nucleolus_enum=tok()

tick()
varcore_dist_enum = var_in_core(enum_mode, OPTIMIZER)  # variance in core
time_elapsed_varcore_enum=tok()

tick()
varleastcore_dist_enum, val_minsurplus_enum, model_dist_enum = var_least_core(
    enum_mode, OPTIMIZER; raw_outputs=true
)  # variance least core (include raw outputs for comparison purposes)
time_elapsed_varleastcore_enum=tok();

Store the EnumMode reward distribution into a DataFrame for ease their use

In [11]:
# vector of the users
user_set_agg = [EC_CODE; get_user_set(ECModel)]

"Auxiliary function to order the output of reward distributions and return them as vectors"
vectorize_rewards(reward_dist, users_list=user_set_agg) = [reward_dist[u] for u in users_list]

# dataframe of reward distributions for the enumerative mode
df_reward_enum = DataFrame(
    user_set=user_set_agg,
    shapley_enum=vectorize_rewards(shapley_dist_enum),
    nucleolus_enum=vectorize_rewards(nucleolus_dist_enum),
    varcore_enum=vectorize_rewards(varcore_dist_enum),
    varleastcore_enum=vectorize_rewards(varleastcore_dist_enum),
)

# dataframe of the time requirements
dict_time_enum = Dict(
    "EnumMode"=>time_elapsed_enum,
    "shapley_enum"=>time_elapsed_shapley_enum+time_elapsed_enum,
    "nucleolus_enum"=>time_elapsed_nucleolus_enum+time_elapsed_enum,
    "varcore_enum"=>time_elapsed_varcore_enum+time_elapsed_enum,
    "varleastcore_enum"=>time_elapsed_varleastcore_enum+time_elapsed_enum,
)

df_reward_enum

,user_set,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum
,Any,Float64,Float64,Float64,Float64
1,EC,1.04939e5,1.09098e5,43112.0,49015.1
2,user4,5248.87,5588.33,10068.7,7954.27
3,user7,22856.2,24636.5,43112.0,46050.6
4,user8,7397.17,4461.78,8923.56,5701.17
5,user9,6298.22,4197.3,8073.94,5172.22
6,user2,5723.51,3222.39,6444.78,3222.39
7,user1,3588.8,3354.85,6351.43,3487.31
8,user5,32604.5,35382.8,43112.0,49015.1
9,user3,6380.4,4234.07,8346.22,5245.75


## Calculate the reward distribution functions by row-generation techniques (IterMode in TheoryOfGames.jl)

### Define the row-generation decomposition mode by IterMode

In [12]:
iter_mode = IterMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP, optimizer=OPTIMIZER_ROW_GENERATION)

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04


Model not optimized


IterMode(Any["EC", "user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "user10"], EnergyCommunity.var"#utility_callback_by_subgroup#662"{EnergyCommunity.var"#objective_callback_by_subgroup#583"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#488"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Any}}, Tuple{JuMP.Containers._AxisLookup{Dict{Any, Int64}}}}}}, EnergyCommunity.var"#objective_callback_by_subgroup#488"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Any}}, Tuple{JuMP.Containers._AxisLookup{Dict{Any, Int64}}}}}}(EnergyCommunity.var"#objective_callback_by_subgroup#583"{ModelEC, EnergyCommunity.var"#objective_callback_by_subgroup#488"{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Any}}, Tuple{JuMP.Containers._AxisLookup{Dict{Any, Int64}}}}}}(An Energy Community Model
Energy Community problem for a Cooperative Model
User set: Any["user4", "user7", "user8", "user9", "user2", "user1", "user5", "user3", "user6", "us

Model not optimized


Define coalitions to precompute when performing the iterative procedure

In [13]:
# include all coalitions having no more than preload_max_size users
preload_combs_set = [1, 2, length(ECModel.user_set)]

preload_coalitions = collect(Iterators.flatten([combinations([EC_CODE; ECModel.user_set], k) for k = preload_combs_set]))

length(preload_coalitions)

77

### Calculate reward distribution using row-generation technique

Variance Least Core using IterMode

In [27]:
tick()
varleastcore_dist_iter, min_surplus_varleastcore_iter, history_varleastcore_iter, model_dist_varleastcore_iter = var_least_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
    best_objective_stop_option="BestObjStop",
)
time_elapsed_varleastcore_iter=tok()
println("Variance Least Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varleastcore_iter/60)")

PHASE 1: Start first least core analysis


┌ Info:  started timer at: 2022-06-24T12:57:35.910
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54



Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only 

1                3.22e+03        3.22e+03        0.00           [194926.1689515598, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983, 3222.387787027983]
PHASE 1: First least core analysis ended with value 3222.387787027983
 ----
PHASE 2: Start second phase to indentify the desired objective

Iteration       Upper bound     Lower bound     Tol. [%]        Benefit distribution


2                3.22e+03        3.22e+03        0.00           [49015.138580672734, 7954.270453938542, 46050.5895173068, 5701.171990191588, 5172.216515587834, 3222.387787027983, 3487.3146465044456, 49015.13858067244, 5245.75016762744, 3270.930001636147, 49015.138580672516]
PHASE 2: Completed second phase

Variance Least Core - IterMode calculated with elapsed time [min]: 11.207293918333333


Variance Core method using IterMode

In [28]:
tick()
varcore_dist_iter, min_surplus_varcore_iter, history_varcore_iter, model_dist_varcore_iter = var_in_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
    best_objective_stop_option="BestObjStop",
)
time_elapsed_varcore_iter=tok()
println("Variance Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varcore_iter/60)")

Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only - expires 2023-02-04
Academic license - for non-commercial use only -

┌ Info:  started timer at: 2022-06-24T13:08:48.462
└ @ TickTock C:\Users\Davide\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


1                0.00e+00       -1.91e+03        100.00         [42634.82268753651, 11176.658240962657, 42634.822687536536, 8923.559777217413, 8394.604302614796, 6444.775574054047, 6709.702433527054, 42634.82268753599, 8468.137954654785, 6493.317788661401, 42634.82268753638]


2                0.00e+00       -2.79e+02        100.00         [43112.01786247298, 9267.877541209426, 43112.01786247268, 8923.559777219763, 8394.604302615748, 6444.775574053219, 6709.702433529633, 43112.01786247277, 8468.137954656171, 6493.3177886634185, 43112.01786247283]


3                0.00e+00       -4.00e+02        100.00         [43112.01786247148, 9546.886807362554, 43112.01786247149, 8923.55977722095, 8394.60430261703, 6444.775574055777, 6430.693167375675, 43112.017862471504, 8468.13795465654, 6493.317788664963, 43112.017862471504]


4                0.00e+00       -2.01e+02        100.00         [43112.01786247162, 9946.814652822626, 43112.01786247166, 8923.559777220766, 7994.676457156902, 6444.775574055653, 6430.693167375927, 43112.017862471635, 8468.137954656482, 6493.317788664957, 43112.017862471606]


5                0.00e+00       -2.99e-09        0.30           [43112.0178624687, 10068.732399335588, 43112.01786246891, 8923.559777226057, 8073.940907863059, 6444.775574059998, 6351.428716669929, 43112.01786246855, 8346.220208144152, 6493.317788668229, 43112.01786246883]
Variance Core - IterMode calculated with elapsed time [min]: 18.03045191333333


Store results as a DataFrame

In [29]:
# dataframe of reward distributions for the enumerative mode
df_reward_iter = DataFrame(
    user_set=user_set_agg,
    varcore_iter=vectorize_rewards(varcore_dist_iter),
    varleastcore_iter=vectorize_rewards(varleastcore_dist_iter),
)

# dataframe of the time requirements
dict_time_iter = Dict(
    "IterMode"=>0.0,
    "varcore_iter"=>time_elapsed_varcore_iter,
    "varleastcore_iter"=>time_elapsed_varleastcore_iter,
)

df_reward_iter

,user_set,varcore_iter,varleastcore_iter
,Any,Float64,Float64
1,EC,43112.0,49015.1
2,user4,10068.7,7954.27
3,user7,43112.0,46050.6
4,user8,8923.56,5701.17
5,user9,8073.94,5172.22
6,user2,6444.78,3222.39
7,user1,6351.43,3487.31
8,user5,43112.0,49015.1
9,user3,8346.22,5245.75


## Group all results and save them

Merge results

In [30]:
df_reward = innerjoin(df_reward_enum, df_reward_iter, on=:user_set)
df_reward

,user_set,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum,varcore_iter,varleastcore_iter
,Any,Float64,Float64,Float64,Float64,Float64,Float64
1,EC,1.04939e5,1.09098e5,43112.0,49015.1,43112.0,49015.1
2,user4,5248.87,5588.33,10068.7,7954.27,10068.7,7954.27
3,user7,22856.2,24636.5,43112.0,46050.6,43112.0,46050.6
4,user8,7397.17,4461.78,8923.56,5701.17,8923.56,5701.17
5,user9,6298.22,4197.3,8073.94,5172.22,8073.94,5172.22
6,user2,5723.51,3222.39,6444.78,3222.39,6444.78,3222.39
7,user1,3588.8,3354.85,6351.43,3487.31,6351.43,3487.31
8,user5,32604.5,35382.8,43112.0,49015.1,43112.0,49015.1
9,user3,6380.4,4234.07,8346.22,5245.75,8346.22,5245.75


In [31]:
dict_time = merge(dict_time_enum, dict_time_iter)
dict_time

Dict{String, Float64} with 8 entries:
  "EnumMode"          => 5091.33
  "varcore_iter"      => 1081.83
  "varleastcore_enum" => 5092.35
  "shapley_enum"      => 5092.08
  "IterMode"          => 0.0
  "varleastcore_iter" => 672.438
  "nucleolus_enum"    => 5170.69
  "varcore_enum"      => 5093.65

Save results

In [32]:
# print to latex the reward table
set_default(fmt = "%.2f", convert_unicode = false)

sorted_user_set = [EC_CODE; ["user$u" for u in 1:length(get_user_set(ECModel))]]

df_reward_mod = copy(df_reward)
sort!(df_reward_mod, [order(:user_set, by=x->findfirst(x .== sorted_user_set))])
df_reward_mod[!, 2:end] = df_reward[!, 2:end] ./ 1000  # change € unit to k€
tex_df_reward = latexify(df_reward_mod; env=:table, latex=false)

L"\begin{tabular}{ccccccc}
user_set & shapley_enum & nucleolus_enum & varcore_enum & varleastcore_enum & varcore_iter & varleastcore_iter\\
EC & 104.94 & 109.10 & 43.11 & 49.02 & 43.11 & 49.02\\
user1 & 5.25 & 5.59 & 10.07 & 7.95 & 10.07 & 7.95\\
user2 & 22.86 & 24.64 & 43.11 & 46.05 & 43.11 & 46.05\\
user3 & 7.40 & 4.46 & 8.92 & 5.70 & 8.92 & 5.70\\
user4 & 6.30 & 4.20 & 8.07 & 5.17 & 8.07 & 5.17\\
user5 & 5.72 & 3.22 & 6.44 & 3.22 & 6.44 & 3.22\\
user6 & 3.59 & 3.35 & 6.35 & 3.49 & 6.35 & 3.49\\
user7 & 32.60 & 35.38 & 43.11 & 49.02 & 43.11 & 49.02\\
user8 & 6.38 & 4.23 & 8.35 & 5.25 & 8.35 & 5.25\\
user9 & 4.87 & 3.25 & 6.49 & 3.27 & 6.49 & 3.27\\
user10 & 27.24 & 29.73 & 43.11 & 49.02 & 43.11 & 49.02\\
\end{tabular}
"

In [33]:
df_time = DataFrame(dict_time)[!, names(df_reward)[2:end]]./3600
df_time[!, "title"] = [L"Time [h]"]
df_time = df_time[!, ["title"; names(df_reward)[2:end]]]

,title,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum,varcore_iter,varleastcore_iter
,LaTeXStr…,Float64,Float64,Float64,Float64,Float64,Float64
1,$Time [h]$,1.41447,1.4363,1.4149,1.41454,0.300508,0.186788


In [34]:
df_iterations = DataFrame(
    title=L"Iterations",
    shapley_enum=L"-",
    nucleolus_enum=n_iterations_nucleolus_enum,
    varcore_enum=L"-",
    varleastcore_enum=L"-",
    varcore_iter=history_varcore_iter[end][1],
    varleastcore_iter=history_varleastcore_iter[end][1],
)

,title,shapley_enum,nucleolus_enum,varcore_enum,varleastcore_enum,varcore_iter,varleastcore_iter
,LaTeXStr…,LaTeXStr…,Int64,LaTeXStr…,LaTeXStr…,Int64,Int64
1,$Iterations$,$-$,2037,$-$,$-$,5,2


In [35]:
df_computational_time = vcat(df_time, df_iterations)

# print to latex the equivalent dataframe for time
tex_df_computational_time = latexify(df_computational_time; env=:table)

L"\begin{tabular}{ccccccc}
$title$ & $shapley_{enum}$ & $nucleolus_{enum}$ & $varcore_{enum}$ & $varleastcore_{enum}$ & $varcore_{iter}$ & $varleastcore_{iter}$\\
$Time [h]$ & $1.41$ & $1.44$ & $1.41$ & $1.41$ & $0.30$ & $0.19$\\
$Iterations$ & $-$ & $2037.00$ & $-$ & $-$ & $5.00$ & $2.00$\\
\end{tabular}
"

In [36]:
# save latex code
open(latex_output,"w") do io
    println(io, "Reward distribution table\n\n\n")
    print(io, tex_df_reward)
    println(io, "Computational time table\n\n\n")
    print(io, tex_df_computational_time)
end

In [37]:
if !isfile(total_results_file) || overwrite_files
    jldsave(total_results_file; df_reward, dict_time, df_computational_time)
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 C:\Users\Davide\.julia\packages\JLD2\k9Gt0\src\JLD2.jl:233
